In [ ]:
!pip install transformers datasets

In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import datasets
from transformers import Trainer, TrainingArguments

In [3]:
tokenizer = AutoTokenizer.from_pretrained("t5-small")
model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

def preprocess_function(examples):
  inputs = examples["text"]
  targets = examples["label"]
  # Convert targets to strings
  targets = [str(target) for target in targets]

  # Tokenize inputs and targets with padding and truncation for consistent lengths
  model_inputs = tokenizer(
      inputs,
      text_target=targets,
      truncation=True,
      padding="max_length",
      max_length=128,
  )
  return model_inputs

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
dataset = datasets.load_dataset("imdb")

In [ ]:
tokenized_dataset = dataset['train'].map(preprocess_function, batched=True)

In [6]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Split the tokenized_dataset into train and test sets
train_dataset = tokenized_dataset.select(range(20000))
eval_dataset = tokenized_dataset.select(range(20000, 25000))
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [7]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ac8736 (ac8736-new-york-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,0.005700,0.007485
2,0.004300,0.004949
3,0.003600,0.004778


TrainOutput(global_step=7500, training_loss=0.15294464747111003, metrics={'train_runtime': 1497.711, 'train_samples_per_second': 40.061, 'train_steps_per_second': 5.008, 'total_flos': 2030127022080000.0, 'train_loss': 0.15294464747111003, 'epoch': 3.0})

The objective of this model is to provide sentiment analysis. The significance of this is that I was able to fine tune a large NLP model without needing a significant amount of resources. For the preparation phase, I just split the data into training and validation, and also made sure to tokenize the data and pad them to be consistent lengths. I think more epochs or more diverse data than help improve the performance of the model.